In [2]:
import time
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup, Tag
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

kbo 홈페이지에서 경기와 선수에 대한 기록을 크롤링하는 코드입니다.

kia를 중심으로 돼 있으며, 수집 후 전처리까지 진행해 놨습니다.

주로 ssg와 롯데를 상대했을 때 기록입니다.

In [16]:
print(f'Python version : {sys.version}')
print(f'Pandas version : {pd.__version__}')
print(f'bs4 version : {bs4.__version__}')
print(f'selenium version : {selenium.__version__}')
print(f'webdriver_manager version : {webdriver_manager.__version__}')

Python version : 3.11.3 | packaged by Anaconda, Inc. | (main, Apr 19 2023, 23:46:34) [MSC v.1916 64 bit (AMD64)]
Pandas version : 2.2.2
bs4 version : 4.12.2
selenium version : 4.31.1
webdriver_manager version : 4.0.2


In [3]:
num_team = [2, 6, 10] # 2는 기아, 6은 ssg, 10은 롯데

def driver_chrome(url : str) : 
    # 기존에 사용하던 Service가 아닌 ChromeService를 사용.
    # driver도 프로그램을 직접 사용하는 것이 아닌 os에 설치한 크롬 웹 브라우저 드라이버를 불러옴
    service = webdriver.ChromeService(executable_path=ChromeDriverManager().install())

    options = webdriver.ChromeOptions()

    driver = webdriver.Chrome(service=service, options=options)

    driver.get(url)
    driver.implicitly_wait(5)

    return driver

def pitcher_vs_hitter(url = 'https://www.koreabaseball.com/Record/Etc/HitVsPit.aspx', time_to_sleep : float = 0.5) : 

    # 투수 영역의 selecotor
    select_pit_area = '#cphContents_cphContents_cphContents_ddlPitcherTeam'
    # 투수팀들의 selector
    select_pit_team = '#cphContents_cphContents_cphContents_ddlPitcherTeam > option:nth-child'
    # 투수 리스트 selector
    select_pit_player = '#cphContents_cphContents_cphContents_ddlPitcherPlayer'
    # 투수 이름 selector
    select_pit_player_name = '#cphContents_cphContents_cphContents_ddlPitcherPlayer > option:nth-child'
    
    # 타자 영역의 selecotor
    select_hit_area = '#cphContents_cphContents_cphContents_ddlHitterTeam'
    # 타자팀들의 selector
    select_hit_team = '#cphContents_cphContents_cphContents_ddlHitterTeam > option:nth-child'
    # 타자 리스트 selector
    select_hit_player = '#cphContents_cphContents_cphContents_ddlHitterPlayer'
    # 타자 이름 selector
    select_hit_player_name = '#cphContents_cphContents_cphContents_ddlHitterPlayer > option:nth-child'

    indicator_list = ['투수팀', '투수', '타자팀', '타자', '타율', '타석', '타수', '안타', '2루타', '3루타', '홈런', '타점', '볼넷', '사구', '삼진', '장타율', '출루율', 'OPS']

    htmls = []

    start_time = time.time()

    driver = driver_chrome(url=url)

    # 상대 투수팀 영역 클릭
    time.sleep(time_to_sleep)
    driver.find_element(by=By.CSS_SELECTOR, value=select_pit_area).click()

    # cnt = 0

    for t1 in num_team :

        cnt_pitcher_player = 2
        # cnt_pitcher_player = 15

        form_pit_team = f'{select_pit_team}({t1})'

        # 상대팀 클릭
        time.sleep(time_to_sleep)
        driver.find_element(by=By.CSS_SELECTOR, value=form_pit_team).click()

        time.sleep(time_to_sleep)
        driver.find_element(by=By.CSS_SELECTOR, value=select_pit_player).click()

        while True : 

            form_pit_player = f'{select_pit_player_name}({cnt_pitcher_player})'
            
            try :
                # 상대팀 투수 클릭
                time.sleep(time_to_sleep)
                driver.find_element(by=By.CSS_SELECTOR, value=form_pit_player).click()

                time.sleep(time_to_sleep)
                driver.find_element(by=By.CSS_SELECTOR, value=select_hit_area).click()

                for t2 in num_team : 
                    if (t1 == 2 and t2 in (6, 10)) | (t1 != 2 and t2 == 2): 
                        
                        cnt_hitter_player = 2
                        form_hit_team = f'{select_hit_team}({t2})'

                        time.sleep(time_to_sleep)
                        driver.find_element(by=By.CSS_SELECTOR, value=form_hit_team).click()

                        time.sleep(time_to_sleep)
                        driver.find_element(by=By.CSS_SELECTOR, value=select_hit_player).click()

                        while True : 
                            try :
                                form_hit_player = f'{select_hit_player_name}({cnt_hitter_player})'

                                time.sleep(time_to_sleep)
                                driver.find_element(by=By.CSS_SELECTOR, value=form_hit_player).click()

                                time.sleep(time_to_sleep)
                                driver.find_element(by=By.CSS_SELECTOR, value='#cphContents_cphContents_cphContents_btnSearch').click()

                                time.sleep(time_to_sleep)
                                htmls.append(driver.page_source)
                                
                                cnt_hitter_player += 1

                            except : 
                                break

                cnt_pitcher_player += 1

            except : 
                print(f'투수 {cnt_pitcher_player}번째 선수는 없습니다.')
                break

                # cnt += 1
        
    # driver 세션 종료
    driver.quit()

    records = []

    for html in tqdm(htmls) :
        record = []

        soup = BeautifulSoup(html, 'html.parser')

        table = soup.select_one('#cphContents_cphContents_cphContents_udpRecord > table > tbody')
        selected_option = soup.findAll('option', selected=True)
        option_list = [s.text for s in selected_option]

        for tr in table : 
            
            for td in tr : 

                if (isinstance(td, Tag)): # td이 type이 Tag인지 확인

                    td_s = td.text.strip()

                    if td_s == '기록이 없습니다.' : 
                        # 없는 기록
                        td_s = ''
                        
                        for i in range(14) : 
                            record.append(td_s)        
                            
                    else : 
                        record.append(td_s)

        record = option_list + record

        # print(record)

        records.append(record)

    df_record_pit_hit = pd.DataFrame(data=records, columns=indicator_list)

    end_time = time.time()

    print(f'투수 vs 타자 크롤링 시간 : {round((end_time - start_time) / 60, 3)}분')

    return df_record_pit_hit


def each_player(url='https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx', time_to_sleep: float = 0.5):
    # Selectors
    selectors = {
        'hitter_area': '#contents > div.sub-content > div.tab-depth2 > ul > li.on > a',
        'pitcher_area': '#contents > div.sub-content > div.tab-depth2 > ul > li:nth-child(2) > a',
        'fielder_area': '#contents > div.sub-content > div.tab-depth2 > ul > li:nth-child(3) > a',
        'running_area': '#contents > div.sub-content > div.tab-depth2 > ul > li.last > a',
        'our_team_area': '#cphContents_cphContents_cphContents_ddlTeam_ddlTeam',
        'our_team_kia': '#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option:nth-child(2)',
        'game_situation_area1': '#cphContents_cphContents_cphContents_ddlSituation_ddlSituation',
        'game_situation_enemy': '#cphContents_cphContents_cphContents_ddlSituation_ddlSituation > option:nth-child(6)',
        'game_situation_area2': '#cphContents_cphContents_cphContents_ddlSituationDetail_ddlSituationDetail',
        'game_situation_enemy_team': '#cphContents_cphContents_cphContents_ddlSituationDetail_ddlSituationDetail > option:nth-child',
        'table_records': '#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody'
    }

    indicators = {
        'hitter': ['선수명', '상대팀', '타율', '타수', '안타', '2루타', '3루타', '홈런', '타점', '볼넷', '사구', '삼진', '병살타'],
        'pitcher': ['선수명', '상대팀', '피안타', '2루타', '3루타', '홈런', '볼넷', '사구', '삼진', '폭투', '보크', '피안타율'],
        'fielder': ['선수명', '포지션', '경기수', '선발경기수', '수비이닝', '실책', '견제사', '풋아웃', '어시스트', '병살', '수비율', '포일', '도루허용', '도루실패', '도루저지율'],
        'running': ['선수명', '경기수', '도루시도', '도루허용', '도루실패', '도루성공률', '주루사', '견제사']
    }

    enemy_list = [n for n in num_team if n != 2]  # SSG와 롯데

    var_list = ['hitter', 'pitcher', 'fielder', 'running']
    htmls = {var: [] for var in var_list}

    start_time = time.time()
    driver = driver_chrome(url=url)

    for s in var_list:
        var_area = selectors[s + '_area']
        time.sleep(time_to_sleep)
        driver.find_element(by=By.CSS_SELECTOR, value=var_area).click()

        time.sleep(time_to_sleep)
        driver.find_element(by=By.CSS_SELECTOR, value=selectors['our_team_area']).click()

        time.sleep(time_to_sleep)
        driver.find_element(by=By.CSS_SELECTOR, value=selectors['our_team_kia']).click()

        if s in ('hitter', 'pitcher'):
            time.sleep(time_to_sleep)
            driver.find_element(by=By.CSS_SELECTOR, value=selectors['game_situation_area1']).click()

            time.sleep(time_to_sleep)
            driver.find_element(by=By.CSS_SELECTOR, value=selectors['game_situation_enemy']).click()

            time.sleep(time_to_sleep)
            driver.find_element(by=By.CSS_SELECTOR, value=selectors['game_situation_area2']).click()

            for e in enemy_list:
                form_enemy_team = f"{selectors['game_situation_enemy_team']}({e})"
                time.sleep(time_to_sleep)
                driver.find_element(by=By.CSS_SELECTOR, value=form_enemy_team).click()

                time.sleep(time_to_sleep)
                htmls[s].append(driver.page_source)
        else:
            time.sleep(time_to_sleep)
            htmls[s].append(driver.page_source)

    driver.quit()

    dataframes = {}

    for v in tqdm(var_list):

        var_html = htmls[v]
        var_indicator = indicators[v]
        records = []

        for html in var_html:

            soup = BeautifulSoup(html, 'html.parser')
            # 기록 테이블
            table = soup.select_one(selectors['table_records'])
            # select의 option
            selected_option = soup.findAll('option', selected=True)
            option_list = [s.text for s in selected_option]
            enemy = option_list[-1] # 상대팀

            for tr in table:

                record = []
                cnt_td = 0

                for td in tr:

                    if (isinstance(td, Tag)): # td이 type이 Tag인지 확인

                        if cnt_td not in (0, 2):
                            
                            record.append(td.text.strip())
                    
                        cnt_td += 1

                if v in ('hitter', 'pitcher'):
                    record.insert(1, enemy)

                if len(record) > 1:
                    records.append(record)

        dataframes[v] = pd.DataFrame(data=records, columns=var_indicator)

    end_time = time.time()
    print(f'선수 기록 크롤링 시간 : {round((end_time - start_time) / 60, 3)}분')

    return dataframes['hitter'], dataframes['pitcher'], dataframes['fielder'], dataframes['running']

def change_type(df, type_data : str) : 
    """
    완성된 데이터프레임 중 숫자 타입으로 변경하고자 합니다.
    type_data(하나 선택) : 타자, 투수, 수비, 주루, 투수_타자
    """

    df_c = df.copy()
    
    if type_data == '투수_타자' : 
        col_int = ['타석', '타수', '안타', '2루타', '3루타', '홈런', '타점', '볼넷', '사구', '삼진']
        col_float = ['타율', '장타율', '출루율', 'OPS']

    elif type_data == '타자' : 
        col_int = ['타수', '안타', '2루타', '3루타', '홈런', '타점', '볼넷', '사구', '삼진', '병살타']
        col_float = ['타율']

    elif type_data == '투수' : 
        col_int = ['피안타', '2루타', '3루타', '홈런', '볼넷', '사구', '삼진', '폭투', '보크']
        col_float = ['피안타율']

    elif type_data == '수비' : 
        # 이닝은 대분수 형태로 표현돼 있으므로, 실수 형태로 변환
        df_c['수비이닝_정수'] = df_c['수비이닝'].str.split(' ')

        df_c['수비이닝_소수점'] = df_c['수비이닝_정수'].apply(lambda x : eval(x[-1]) if len(x) > 1 else 0) # eval은 텍스트 형태의 계산식을 계산해 줌

        df_c['수비이닝_정수'] = df_c['수비이닝_정수'].apply(lambda x : eval(x[0]))

        df_c['수비이닝'] = df_c['수비이닝_정수'] + df_c['수비이닝_소수점']

        df_c = df_c[[c for c in df_c.columns if c not in ('수비이닝_정수', '수비이닝_소수점')]]

        col_int = ['경기수', '선발경기수', '실책', '견제사', '풋아웃', '어시스트', '병살', '포일', '도루허용', '도루실패']
        col_float = ['수비율', '도루저지율']

    elif type_data == '주루' : 
        col_int = ['경기수', '도루시도', '도루허용', '도루실패', '주루사', '견제사']
        col_float = ['도루성공률']

    df_c = df_c.replace({'' : 0, '-' : 0})

    df_c[col_int] = df_c[col_int].astype(int)
    df_c[col_float] = df_c[col_float].astype(float)

    return df_c


In [ ]:
if __name__ == '__main__' : 

    # 투수 vs 타자 기록
    record_pit_hit = pitcher_vs_hitter(time_to_sleep=0.6)
    record_pit_hit = change_type(df=record_pit_hit, type_data='투수_타자')

    # 타자, 투수, 수비, 주루 기록
    hitter, pitcher, fielder, running = each_player(time_to_sleep=0.7)
    hitter = change_type(df=hitter, type_data='타자')
    pitcher = change_type(df=pitcher, type_data='투수')
    fielder = change_type(df=fielder, type_data='수비')
    running = change_type(df=running, type_data='주루')

    # csv 저장
    record_pit_hit.to_csv('./data/record_pit_hit.csv', header=True, index=False)
    hitter.to_csv('./data/hitter.csv', header=True, index=False)
    pitcher.to_csv('./data/pitcher.csv', header=True, index=False)
    fielder.to_csv('./data/fielder.csv', header=True, index=False)
    running.to_csv('./data/running.csv', header=True, index=False)